# Phys331 Lab Faraday Effect and Dispersion in Liquids

Here is the python package I used.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit
%matplotlib inline
import matplotlib as mpl

1. Data preprocessing

In [4]:
df = pd.read_csv("fara_green.csv")
df.head()

,Current,sig_current,Angle_of_extinction,sig_degree
0,-1000.0,1.0,95.167,0.083
1,-800.0,1.0,94.333,0.083
2,-601.0,1.0,93.417,0.083
3,-399.0,1.0,92.167,0.083
4,-200.0,1.0,91.167,0.083
